# Getting Started with Landsat Surface Temperature

This notebook is a very simple example of the fundamentals of working with
Earth observation data using the Open Data Cube.

## Import libraries

In [ ]:
from datacube import Datacube
from datacube.utils.aws import configure_s3_access
from datacube.utils import masking
from odc.geo.geom import point

from utils import patch_usgs_landsat

## Configure the environment

In [ ]:
configure_s3_access(requester_pays=True)

dc = Datacube()

## Pick a study area

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
coords = -8.6252, 115.2048  # Denpasar, Bali
aoi_point = point(coords[1], coords[0], crs="EPSG:4326")
bbox = aoi_point.buffer(0.05).boundingbox

landsat_stretch = dict(vmin=7500, vmax=12000)

datetime = "2025"

# Preview the area
bbox.explore(zoom=8)

## Load data

This uses the Datacube library to handle loading of the actual data. The `dask_chunks` argument instructs the tool to use Dask
to lazy-load the data.

Here we mask out clouds, which interfere with the temperature, and then convert to degrees celcius.

In [ ]:
data = dc.load(
    product="ls9_c2l2_st",
    measurements=["st", "qa_pixel"],
    output_crs="EPSG:32750",
    resolution=30,
    time=datetime,
    longitude=(bbox.left, bbox.right),
    latitude=(bbox.bottom, bbox.top),
    dask_chunks={"time": 1, "x": 512, "y": 512},
    group_by="solar_day",
    patch_url=patch_usgs_landsat
)

# Select clouds
mask, _ = masking.create_mask_value(
    data["qa_pixel"].attrs["flags_definition"], cloud="high_confidence", cloud_shadow="high_confidence"
)

pq_mask = (data["qa_pixel"] & mask) != 0
nodata = data.st == 0
mask = pq_mask | nodata

# Combine the masks
data = data.where(~mask)

data["st"] = data.st * 0.00341802 + 149.0 - 273.15

data

## Visualise data

This step uses `matplotlib` to view data as a static image. It takes a longer time to
run than previous steps, because it's actually loading the data to prepare the images.

The `to_array()` function is a trick used to be able to visualise the data as a
red, green, blue "true colour" image.

In [ ]:
data.st.plot.imshow(col="time", col_wrap=2, size=4, cmap="coolwarm")

In [ ]:
# Get a spatial mean and plot a timeseries
clean = data.st.dropna(dim="time", how="all")
mean = clean.mean(dim=("x", "y")).compute()
mean.plot.line(x="time", size=4, color="black", label="Mean")

## Export data

Here we write data to disk, again using an `odc` tool.

In [ ]:
rgba = best.odc.to_rgba(**landsat_stretch)
rgba.odc.write_cog("landsat_example.tif", overwrite=True)